In [1]:
import os
import numpy as np
import torch
import pandas as pd
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, iirnotch, periodogram
from sklearn.preprocessing import StandardScaler
from scipy.stats import kurtosis
from torch.utils.data import random_split
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import gc
from sklearn.metrics import r2_score
from scipy.io import savemat
import joblib
from scipy.signal import hilbert
from sklearn.decomposition import PCA
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from typing import Literal

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", device)

2025-05-22 13:48:25.669158: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-22 13:48:25.770727: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747936105.810845   43765 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747936105.823171   43765 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747936105.910230   43765 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Using Device: cuda


In [17]:
# Read in the data
processed_data_l_X = glob(os.path.join('/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/', "**", "X.npy"))
processed_data_l_y = glob(os.path.join('/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/', "**", "y.npy"))


In [18]:
processed_data_l_X

['/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S7)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-24_(S10)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-04-29_(S1)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-06_(S6)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S5)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S3)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S4)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S8)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-17_(S9)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-04-29_(S2)/X.npy']

In [21]:
# Load session paths
processed_data_l_X = sorted(glob('/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/**/X.npy', recursive=True))
processed_data_l_y = sorted(glob('/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/**/y.npy', recursive=True))


In [ ]:
processed_data_l_X[0]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-04-29_(S1)/X.npy'

In [ ]:
# Define K-fold sets
test_list_X = []
train_list_X = []
test_list_y = []
train_list_y = []

for i in range(len(processed_data_l_X)):
    test_list_X.append(processed_data_l_X[i])
    test_list_y.append(processed_data_l_y[i])
    train_X = [x for idx, x in enumerate(processed_data_l_X) if idx != i]
    train_y = [y for idx, y in enumerate(processed_data_l_y) if idx != i]
    train_list_X.append(train_X)
    train_list_y.append(train_y)

In [20]:
train_list_X[0]

['/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-24_(S10)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-04-29_(S1)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-06_(S6)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S5)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S3)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S4)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S8)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-17_(S9)/X.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-04-29_(S2)/X.npy']

In [9]:
test_list_y

['/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S7)/y.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-24_(S10)/y.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-04-29_(S1)/y.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-06_(S6)/y.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S5)/y.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S3)/y.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S4)/y.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S8)/y.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-17_(S9)/y.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-04-29_(S2)/y.npy',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-03-15_(S1)/y.npy',
 '/home/linux-pc/g

In [ ]:
# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Training and Test data
# Read in the data
processed_data_l_X = glob(os.path.join('/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/', '**', "**", "X.npy"))
processed_data_l_y = glob(os.path.join('/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/', '**', "**", "y.npy"))
# Define K-fold sets
test_list_X = []
train_list_X = []
test_list_y = []
train_list_y = []

for i in range(len(processed_data_l_X)):
    test_list_X.append(processed_data_l_X[i])
    test_list_y.append(processed_data_l_y[i])
    train_X = [x for idx, x in enumerate(processed_data_l_X) if idx != i]
    train_y = [y for idx, y in enumerate(processed_data_l_y) if idx != i]
    train_list_X.append(train_X)
    train_list_y.append(train_y)

# Load a single specific dataset
# K-Fold 0 uses session 7 as the test set:
KFOLD = 0
SESSION_SET = 6

"""
['/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-24_(S10)/X.npy', 0
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-04-29_(S1)/X.npy', 1
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-06_(S6)/X.npy', 2
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S5)/X.npy', 3
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S3)/X.npy', 4
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-03_(S4)/X.npy', 5
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S8)/X.npy', 6 
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-17_(S9)/X.npy', 7
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-04-29_(S2)/X.npy'] 8
"""

for session in range(len(train_list_X[KFOLD])):
X = np.load(train_list_X[KFOLD][SESSION_SET])
y = np.load(train_list_y[KFOLD][SESSION_SET])

# X = np.load(test_list_X[0]) # Identify the test set
# y = np.load(test_list_y[0]) # Identify the test set

# Creating Train and Validation Sets
dataset = EcogMotionDataset(X, y)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)

# Assuming train_loader, val_loader, criterion are defined

# 2. CNN_LSTM Hybrid Model
hybrid_model = EcogToMotionNet()
criterion = nn.MSELoss()
hybrid_train_losses, hybrid_val_losses, hybrid_r2 = train_model(hybrid_model, train_loader, val_loader,device, epochs=100, model_name="Hybrid_CNN_LSTM")


In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader, ConcatDataset
from torch.utils.tensorboard import SummaryWriter
from glob import glob
from model import EcogToMotionNet
from data_loader import EcogMotionDataset
from train import train_model  # assumes it returns (train_losses, _, r2_train)
from sklearn.metrics import r2_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load paths to data
processed_data_l_X = sorted(glob(os.path.join('/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/', '**', '**', "X.npy"), recursive=True))
processed_data_l_y = sorted(glob(os.path.join('/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/', '**', '**', "y.npy"), recursive=True))

# Construct leave-one-session-out splits
test_list_X, train_list_X, test_list_y, train_list_y = [], [], [], []

for i in range(len(processed_data_l_X)):
    test_list_X.append(processed_data_l_X[i])
    test_list_y.append(processed_data_l_y[i])
    train_X = [x for idx, x in enumerate(processed_data_l_X) if idx != i]
    train_y = [y for idx, y in enumerate(processed_data_l_y) if idx != i]
    train_list_X.append(train_X)
    train_list_y.append(train_y)

# Directory to save checkpoints and logs
os.makedirs("checkpoints", exist_ok=True)
os.makedirs("runs", exist_ok=True)

# Training loop across all folds
results = []

for KFOLD in range(len(test_list_X)):
    print(f"=== Fold {KFOLD} ===")

    # Setup TensorBoard SummaryWriter for this fold
    writer = SummaryWriter(log_dir=f"runs/fold_{KFOLD}")

    # Load training datasets (all sessions except the test one)
    train_datasets = []
    for session_idx in range(len(train_list_X[KFOLD])):
        X = np.load(train_list_X[KFOLD][session_idx])
        y = np.load(train_list_y[KFOLD][session_idx])
        train_datasets.append(EcogMotionDataset(X, y))

    full_train_dataset = ConcatDataset(train_datasets)
    train_loader = DataLoader(full_train_dataset, batch_size=64, shuffle=True)

    # Load test dataset (the left-out session)
    X_test = np.load(test_list_X[KFOLD])
    y_test = np.load(test_list_y[KFOLD])
    test_dataset = EcogMotionDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=64)

    # Model and Loss
    model = EcogToMotionNet().to(device)
    criterion = torch.nn.MSELoss()

    # Train model
    train_losses, _, r2_train = train_model(model, train_loader, None, device, epochs=100, model_name=f"fold_{KFOLD}")

    # Log training loss to TensorBoard
    for epoch, loss in enumerate(train_losses):
        writer.add_scalar("Loss/Train", loss, epoch)

    # Evaluate on test set
    model.eval()
    total_loss = 0
    total_r2 = 0
    total_samples = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device).float()
            targets = targets.to(device).float()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)
            total_r2 += r2_score(targets.cpu().numpy(), outputs.cpu().numpy()) * inputs.size(0)
            total_samples += inputs.size(0)

    avg_loss = total_loss / total_samples
    avg_r2 = total_r2 / total_samples
    print(f"Fold {KFOLD} - Test Loss: {avg_loss:.4f}, R^2: {avg_r2:.4f}")

    # Log evaluation metrics
    writer.add_scalar("Loss/Test", avg_loss, 0)
    writer.add_scalar("R2/Test", avg_r2, 0)

    # Save model checkpoint
    checkpoint_path = f"checkpoints/model_fold_{KFOLD}.pt"
    torch.save({
        'model_state_dict': model.state_dict(),
        'test_loss': avg_loss,
        'test_r2': avg_r2
    }, checkpoint_path)

    writer.close()
    results.append((KFOLD, avg_loss, avg_r2))

# After all folds, print summary
print("\n=== Summary Across All Folds ===")
for fold, loss, r2 in results:
    print(f"Fold {fold} | Test Loss: {loss:.4f} | R^2: {r2:.4f}")